In [3]:
import os
import pandas as pd
import os, sys, json
from datetime import date
import parsl
from parsl.app.app import python_app
import subprocess
import argparse


In [15]:
whereis_script = '/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/TFpred_pipeline/test_run/' #os.path.dirname(sys.argv[0]) # or os.path.dirname(__file__)
os.chdir(whereis_script), os.getcwd()

(None,
 '/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/TFpred_pipeline/test_run')

In [16]:

mtdata = '../metadata/aggregation_config_freedman_FOXA1.json'

agg_type = 'aggByMean'

script_path = '../scripts'
utils_path = os.path.join(script_path, 'utilities')
sys.path.append(utils_path)

In [17]:
with open(f'{mtdata}') as f:
    parameters = json.load(f)

    enformer_predictions_path = parameters['enformer_prediction_path']
    log_file = parameters['log_file']
    dataset_type = parameters['dataset_type']# e.g. "kawakami" or "cistrome"
    todays_date = parameters['run_date']
    base_path = parameters['predictions_folder']
    TF = parameters['transcription_factor']
    individuals = parameters['individuals']

In [34]:
print(f'[INFO] Currently on {dataset_type}')

save_dir = f'{base_path}/aggregated_predictions/{dataset_type}_{TF}_{todays_date}'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

if individuals is None:
    ids_names = [dataset_type]
elif isinstance(individuals, list):
    ids_names = individuals

log_data_all = pd.read_csv(log_file)

# use parsl if the num of rows of log_data is more than 10000
use_parsl = False

predict_utils_one = f'{script_path}/utilities/utility_functions.py'
exec(open(predict_utils_one).read(), globals(), globals())

if use_parsl == True:
    #bpath = os.path.join(base_path, 'modeling_pipeline')
    print(f'[INFO] Using parsl. Aggregation will be split into multiple files, and batches will be collected into one later.')
    parsl_params = {'working_dir':base_path, 'job_name':'aggregate_predictions', 'queue':"preemptable", 'walltime':"05:59:00", 'num_of_full_nodes':1, 'min_num_blocks':0, 'max_num_blocks':10}
    parsl.load(parslConfiguration.localParslConfig_htpool(parsl_params))

collection_fxn = return_prediction_function(use_parsl)

[INFO] Currently on freedman


In [27]:
log_data_all = pd.read_csv(log_file)
log_data_all.drop_duplicates(subset=['motif'])

,motif,individual,status,sequence_type
0,chr12_92688598_92688607,LuCaP_136,completed,var
1,hr9_122819978_122819987,LuCaP_136,completed,var
2,chr9_18363310_18363319,LuCaP_136,completed,var
3,chr5_160240472_160240481,LuCaP_136,completed,var
4,chr20_11244716_11244725,LuCaP_136,completed,var
...,...,...,...,...
9099,chrX_68854754_68854763,LuCaP_141,completed,var
9100,chr9_122819978_122819987,LuCaP_141,completed,var
9101,chr7_3576376_3576385,LuCaP_141,completed,var
9174,chr1_176593308_176593317,LuCaP_141,completed,var


In [32]:
log_data_all.shape

(9103, 4)

In [36]:
ids_names = ['LuCaP_141']

for each_id in ids_names:
    log_data = log_data_all.loc[log_data_all['individual'] == each_id, ]
    log_data = log_data.drop_duplicates(subset=['motif']) #.iloc[1:1000, ]

    print(log_data)

    print(log_data.shape)

    range_batches = generate_batch(range(0, log_data.shape[0]), batch_n=8)

    ind_path = os.path.join(enformer_predictions_path, each_id)

    app_futures = []
    for i, range_batch in enumerate(range_batches):

        print(list(range_batches))
        app_futures.append(f'{i}: {len(range_batch)}')

        # print(i)
        # app_futures.append(collection_fxn(each_id=each_id, log_data=log_data.iloc[range_batch, ], predictions_path=ind_path, TF=TF, agg_types=[agg_type], base_path=base_path, save_dir=save_dir, batch_num=i))

    print(app_futures)

    if use_parsl == True:
        app_execs = [r.result() for r in app_futures]

    # cmd = f"for i in `ls {save_dir}/{each_id}_{agg_type}_{TF}_batch_*.csv.gz`; do zcat $i | sed '1d'; done | pigz -c >{save_dir}/{each_id}_{agg_type}_{TF}.csv.gz"    
    # p = subprocess.Popen(cmd, shell=True)
    # print(f'[INFO] Status: Aggregation complete for {log_data.shape[0]} predictions for {each_id}.')

                          motif individual     status sequence_type
9098    chr12_92688598_92688607  LuCaP_141  completed           var
9099     chrX_68854754_68854763  LuCaP_141  completed           var
9100   chr9_122819978_122819987  LuCaP_141  completed           var
9101       chr7_3576376_3576385  LuCaP_141  completed           var
9102     chr3_24951897_24951906  LuCaP_141  completed           var
...                         ...        ...        ...           ...
18195   chr12_23533600_23533609  LuCaP_141  completed           var
18196  chr5_102611585_102611594  LuCaP_141  completed           var
18197    chr8_84108965_84108974  LuCaP_141  completed           var
18198   chr21_24828757_24828766  LuCaP_141  completed           var
18199    chr4_79041264_79041273  LuCaP_141  completed           var

[9102 rows x 4 columns]
(9102, 4)
[range(1138, 2276), range(2276, 3414), range(3414, 4552), range(4552, 5690), range(5690, 6828), range(6828, 7966), range(7966, 9102)]
['0: 1138']
